In [1]:
library(dplyr)
#library(tictoc)
library(lubridate)
library(tidyr)
library(ggplot2)
#library(tsibble)
#library(feasts)
#library(latex2exp)
#library(fUnitRoots)
#library(fable)
#library(fabletools)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Etc/UCT’ (or equivalent)”

Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [2]:
df <- read.csv("../Data/standardised_dt_df.csv")
#df <- readRDS('../Data/Full_data.rds')
df <- df %>% mutate(chunk = as.POSIXct(chunk, format="%Y-%m-%d %H:%M:%S",tz="GMT"))
head(df)

,X,chunk,abnormal,X1.hr,X2.hr,X3.hr,X4.hr,X5.hr,X6.hr,X7.hr,⋯,X19.hr,X20.hr,X21.hr,X22.hr,X23.hr,X24.hr,X25.hr,X26.hr,X27.hr,X28.hr
,<int>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,29,2018-10-26 19:00:00,0,-0.6010679,-1.3726190,-0.0247520,-0.4374796,0.7108556,1.8196562,0.3108767,⋯,-0.5509926,0.3432454,-0.5376402,0.1196872,0.7105955,-0.9455626,-0.9937495,-1.4766641,-2.2057631,-1.4095363
2,30,2018-10-26 20:00:00,0,0.4171159,-0.6010679,-1.3726190,-0.0247520,-0.4374796,0.7108556,1.8196562,⋯,0.1497921,-0.5509926,0.3432454,-0.5376402,0.1196872,0.7105955,-0.9455626,-0.9937495,-1.4766641,-2.2057631
3,31,2018-10-26 21:00:00,0,-0.4045199,0.4171159,-0.6010679,-1.3726190,-0.0247520,-0.4374796,0.7108556,⋯,-0.0304761,0.1497921,-0.5509926,0.3432454,-0.5376402,0.1196872,0.7105955,-0.9455626,-0.9937495,-1.4766641
4,32,2018-10-26 22:00:00,0,-0.9145045,-0.4045199,0.4171159,-0.6010679,-1.3726190,-0.0247520,-0.4374796,⋯,-0.6507914,-0.0304761,0.1497921,-0.5509926,0.3432454,-0.5376402,0.1196872,0.7105955,-0.9455626,-0.9937495
5,33,2018-10-26 23:00:00,0,-0.3375264,-0.9145045,-0.4045199,0.4171159,-0.6010679,-1.3726190,-0.0247520,⋯,1.4907880,-0.6507914,-0.0304761,0.1497921,-0.5509926,0.3432454,-0.5376402,0.1196872,0.7105955,-0.9455626
6,34,2018-10-27 00:00:00,0,1.0419964,-0.3375264,-0.9145045,-0.4045199,0.4171159,-0.6010679,-1.3726190,⋯,-1.1927199,1.4907880,-0.6507914,-0.0304761,0.1497921,-0.5509926,0.3432454,-0.5376402,0.1196872,0.7105955


In [3]:
tail(df)

,chunk,texts,temp_weighted,cloud_cover_weighted,rainfall_weighted,polarity,standardised_texts,abnormal,daily_policy_index,1_hr_std,⋯,19_hr,20_hr,21_hr,22_hr,23_hr,24_hr,25_hr,26_hr,27_hr,28_hr
,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
31681,2022-06-06 15:00:00,70,15.62318,65.29326,0.00000000,0.126058810840715,-0.09154173,0,237.37,-0.05332543,⋯,204,159,113,100,99,89,75,82,67,41
31682,2022-06-06 16:00:00,89,15.69936,68.28912,0.11110657,0.126058810840715,0.57388690,0,237.37,-0.09154173,⋯,190,204,159,113,100,99,89,75,82,67
31683,2022-06-06 17:00:00,104,15.57503,65.45153,0.04970403,0.126058810840715,1.25701812,0,237.37,0.57388690,⋯,222,190,204,159,113,100,99,89,75,82
31684,2022-06-06 18:00:00,124,15.46761,65.15089,0.00964676,0.126058810840715,1.72717183,0,237.37,1.25701812,⋯,230,222,190,204,159,113,100,99,89,75
31685,2022-06-06 19:00:00,140,15.05260,62.54760,0.03686618,0.126058810840715,0.85250885,0,237.37,1.72717183,⋯,181,230,222,190,204,159,113,100,99,89
31686,2022-06-06 20:00:00,188,14.23738,60.13144,0.00000000,0.126058810840715,1.29353656,0,237.37,0.85250885,⋯,110,181,230,222,190,204,159,113,100,99


In [4]:
tstraintest <- function(ts, trainstart, teststart, testend, ex) {
  train <- ts %>% dplyr::filter(chunk >= trainstart & chunk < teststart)
  test <- ts %>% dplyr::filter(chunk >= teststart & chunk < testend)
  if (ex == "expost") {
    print("expost set complete")
  } else if (ex == "exante") {
    # Get average twitter and covid values from last 24 hrs
    twitter_covid <- train[(dim(train)[1]-24):dim(train)[1],] %>% as.data.frame %>% # last 24 hrs
      mutate(year = floor_date(chunk, "year")) %>%
      group_by(year) %>% summarise(tweet_avg = mean(tweets),
                                   retweet_avg = mean(retweets),
                                   likes_avg = mean(likes),
                                   covid_cases = mean(covid_cases),
                                   covid_deaths = mean(covid_deaths),
                                   covid_announcements = mean(covid_announcements))
    # Get average weather from full history prior to test set
    weather <- read_csv("openweather_2010-_2021_snarestone_uk.csv") %>% dplyr::select(dt,dt_iso, temp, clouds_all, rain_1h)
    weather$chunk <- as.POSIXct(weather$dt, tz = "GMT", origin="1970-01-01")
    weather <- weather %>% transmute(chunk, temp, clouds = clouds_all, rainfall = rain_1h) %>% dplyr::filter(chunk<teststart)
    weather$rainfall <- ifelse(is.na(weather$rainfall),0,weather$rainfall)
    weather <- weather %>% mutate(daymonthhour = paste(substr(floor_date(chunk, "days"),9,10),substr(floor_date(chunk, "months"),6,7),substr(chunk,12,13) ))
     
    hourly <- weather %>% group_by(daymonthhour) %>% summarise(temp = mean(temp),
                                                               clouds = mean(clouds),
                                                               rainfall = mean(rainfall),)
    daily_temp <- weather %>% mutate(day = paste(substr(floor_date(chunk, "days"),9,10),substr(floor_date(chunk, "months"),6,7)))
    daily_temp <- daily_temp %>% group_by(day) %>% summarise(daily_temp = mean(temp))
    rm(weather)
    
    # Merge into a test set
    base_test <- test %>% dplyr::select(chunk,texts,localtime,weekday,school_hols,exam_results_week,bank_holiday)
    rm(test)
    base_test <- base_test %>% mutate(daymonthhour = paste(substr(floor_date(chunk, "days"),9,10),substr(floor_date(chunk, "months"),6,7),substr(chunk,12,13) ))
    base_test <- merge(base_test,hourly, by="daymonthhour",all.x=TRUE) %>% dplyr::select(-daymonthhour)
    rm(hourly)
    base_test <- base_test %>% mutate(day = paste(substr(floor_date(chunk, "days"),9,10),substr(floor_date(chunk, "months"),6,7)))
    base_test <- merge(base_test,daily_temp, by="day",all.x=TRUE) %>% dplyr::select(-day)
    rm(daily_temp)
    base_test <- base_test %>% mutate(year = floor_date(chunk, "year"))
    base_test <- merge(base_test,twitter_covid, by="year",all.x=TRUE) %>% dplyr::select(-year)
    rm(twitter_covid)
    test <- as_tsibble(base_test,index=chunk)
    rm(base_test)
    
    train <- train %>% dplyr::select(-c(tweets,retweets,likes))
    
    print("exante set complete")
  } else {
    print("ex command is invalid. Please select 'expost' or 'exante' as an input. Expost simply splits into train and test whereas exante only includes simple forecasts for exogenous regressors.")
  }
  train$traintest <- rep(0, dim(train)[1])
  test$traintest <- rep(1, dim(test)[1])
  output <- rbind(train, test)
  return(output)
}

In [4]:
#Standard train test split
train_test <- sample(c("train","test"), nrow(df), replace=TRUE, prob=c(0.7,0.3))
length(train_test)
df_standard_random <- df
df_standard_random$train_test <- train_test

[1] 31658

In [5]:
write.csv(df_standard_random, "../Data/Full_train_test_split/df_full_random_split.csv")

In [3]:
df_standardised_end_test <- df
df_standardised_end_test$train_test <- ifelse(df_standardised_end_test$chunk <= "2021-12-31 23:00:00", "train","test")

In [6]:
write.csv(df_standardised_end_test, "../Data/Full_train_test_split/df_full_end_test.csv")

In [4]:
#write.csv(df_standardised_end_test, "../Data/df_standardised_end_test_6hr.csv")

In [4]:
write.csv(df_standardised_end_test, "../Data/Full_train_test_split/df_full_end_test_classification.csv")